<a href="https://colab.research.google.com/github/fberanizo/spelling-correction/blob/master/Correcao_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecção e correção de erro ortográfico com modelo T5

**Nome: Fabio Beranizo Lopes**<br>
**Nome: Luiz Pita Almeida**

Usaremos o modelo T5 pré-treinado e o dataset Paracrawl Inglês-Português. <br>
Truncamos para strings de tamanho 100 para deixar os testes mais rápidos.

Passos:

1. Pressupõe que as frases to Paracrawl estão corretas.<br>
2. Aplica-se todas as técnicas para corromper amostras:
   - Drop
   - Swap
   - Add
   - Key
3. Treina-se o modelo T5 com as frases corrompidas como entrada e frase corrigida (original) como saída.

**Obs: os notebooks contém excertos de códigos dos colegas de turma.**<br>
**Obrigado Diedre, Leard e Gabriela.**


In [1]:
import torch

print(f"Current GPU: {torch.cuda.get_device_name(0)}")

# don't even start if it's not a P100 GPU
# if torch.cuda.get_device_name(0) != "Tesla P100-PCIE-16GB":
#     import os
#     os.kill(os.getpid(), 9)

Current GPU: Tesla V100-SXM2-16GB


In [2]:
#@title Configurações gerais
experiment_name = "adversarial-misspellings"  #@param {type:"string"}
model_name = "t5-small"  #@param ["t5-small", "t5-base", "t5-large", "t5-3B", "t5-11B"] {type:"string"}
batch_size = 100  #@param {type:"integer"}
accumulate_grad_batches = 1  #@param {type:"integer"}
sequence_length = 100  #@param {type:"integer"}
learning_rate = 5e-3  #@param {type:"number"}
decode_mode = "topk"  #@param ["greedy", "nucleus", "topk", "beam"] {type:"string"}
k = 10  #@param {type:"integer"}
max_epochs = 1  #@param {type:"integer"}

## Instala dependências

- PyTorch Lightning
- Hugginface Transformers
- ERRANT (ERRor ANnotation Toolkit)

In [3]:
!git clone --quiet https://github.com/danishpruthi/Adversarial-Misspellings.git

try:
    import pytorch_lightning
    import transformers
except (ImportError, ModuleNotFoundError) as e:
    pass
    # can't import modules, then install
    # !pip install --quiet pytorch-lightning
    # !pip install --quiet transformers
    # !pip install --quiet errant==2.2.0
    # !pip install pyxDamerauLevenshtein
    # !python -m spacy download en
    # # kill kernel (necessary for tqdm)
    # import os
    # os.kill(os.getpid(), 9)

fatal: destination path 'Adversarial-Misspellings' already exists and is not an empty directory.


In [4]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.
import datetime
import errant
import gzip
import json
import numpy as np
import nvidia_smi
import os
import pandas as pd
import joblib
import psutil
import pytorch_lightning as pl
import random
import spacy
import sys
import tarfile
import tempfile
import torch
import torch.nn.functional as F

from argparse import Namespace
from collections import deque
# from google.colab import drive
from itertools import cycle
from tqdm import tqdm_notebook

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from typing import Dict
from typing import List
from typing import Tuple

# Leard decoding solution
import html
import unicodedata

nlp = spacy.load("en")
annotator = errant.load("en", nlp)

import nltk
nltk.download("stopwords")

sys.path.insert(0, "/content/Adversarial-Misspellings")
import attacks

nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)

def hardware_stats():
    """
    Returns a dict containing some hardware related stats
    """
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    return {"cpu": f"{str(psutil.cpu_percent())}%",
            "mem": f"{str(psutil.virtual_memory().percent)}%",
            "gpu": f"{str(res.gpu)}%",
            "gpu_mem": f"{str(res.memory)}%"}

[nltk_data] Downloading package stopwords to /home/fabiol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Define random seeds

Importante: Fix seeds so we can replicate results

In [5]:
import random

seed = 0
random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Mapeia Google Drive

Iremos salvar os checkpoints (pesos do modelo) no google drive, para que possamos continuar o treino de onde paramos.

In [6]:
# drive.mount("/content/drive")
# base_path = "/content/drive/My Drive/PF-Correcao/paracrawl-t5"
base_path = "/content/paracrawl-t5"
os.environ["BASE_PATH"] = base_path

## Classe Dataset
Gerenciamento dos dados, e um pequeno teste.
No getitem é aplicada a correção de codificação.

In [7]:
hparams = {"model_name": model_name, "seq_len": sequence_length, "batch_size": batch_size}
class ParaCrawl(Dataset):
    """
    Loads data from preprocessed file and manages them.
    """
    VALID_MODES = ["train", "validation", "test"]
    TOKENIZER = T5Tokenizer.from_pretrained(hparams["model_name"],
                                            cache_dir=base_path)
    def __init__(self, mode: str, seq_len: int):
        """
        mode: One of train, validation or test 
        seq_len: limit to returned encoded tokens
        """
        super().__init__()
        assert mode in ParaCrawl.VALID_MODES

        self.mode = mode
        self.seq_len = seq_len

        file_name = os.path.join(base_path, f"{mode}.pkl")
        if not os.path.isfile(file_name):
            print("Pre-processed files not found, preparing data.")
            self.prepare_data()
        
        with open(file_name, "rb") as preprocessed_file:
            self.data = joblib.load(preprocessed_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i: int):
        """
        Unpacks line from data.

        returns: input (corrputed), target (corrected)
        """
        input, target = self.data[i]

        input_normalized = unicodedata.normalize("NFD", input).\
            encode("latin-1", "xmlcharrefreplace").\
            decode("latin-1")

        input_encoded = ParaCrawl.TOKENIZER.encode_plus(
            f"{input_normalized} {ParaCrawl.TOKENIZER.eos_token}",
            max_length=self.seq_len,
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_tensors="pt")

        input_ids = input_encoded["input_ids"].squeeze()
        input_mask = input_encoded["attention_mask"].squeeze()

        target_normalized = unicodedata.normalize("NFD", target).\
            encode("latin-1", "xmlcharrefreplace").\
            decode("latin-1")

        target_encoded = ParaCrawl.TOKENIZER.encode_plus(
            f"{target_normalized} {ParaCrawl.TOKENIZER.eos_token}",
            max_length=self.seq_len,
            pad_to_max_length=True,
            return_token_type_ids=False,
            return_tensors="pt")

        target_ids = target_encoded["input_ids"].squeeze()
        target_mask = target_encoded["attention_mask"].squeeze()

        return input, target, input_ids, input_mask, target_ids, target_mask

    def get_dataloader(self, batch_size: int, shuffle: bool):
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, 
                          num_workers=4)

    @staticmethod
    def load_text_pairs(path):
        """
        Load pairs from original files, selects pt, then corrupts the samples.
        """
        text_pairs = []
        for line in tqdm_notebook(gzip.open(path, mode="rt")):
            text_pt = line.strip().split("\t")[1]
            text_pt = text_pt[:hparams["seq_len"]].rsplit(" ", 1)[0]
            try:
                attack_list = deque(attacks.all_one_attack(text_pt, include_ends=True))
                text_corrupt = list(map(lambda a: a[1], random.sample(attack_list, k=10)))
                text_pairs.extend(list(zip(text_corrupt, cycle([text_pt]))))
            except:
                pass

        return text_pairs

    @staticmethod
    def prepare_data(train_size=9997000, val_size=3000):
        """
        Performs everything needed to get the data ready.
        Addition of Eos token and encoding is performed in runtime.
        """
        if not os.path.isfile("paracrawl_enpt_train.tsv.gz"):    
            !wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_train.tsv.gz -P "$BASE_PATH"
            !wget -nc https://storage.googleapis.com/neuralresearcher_data/unicamp/ia376e_2020s1/paracrawl_enpt_test.tsv.gz -P "$BASE_PATH"

        data = {}
        test_data = ParaCrawl.load_text_pairs(os.path.join(base_path, "paracrawl_enpt_test.tsv.gz"))
        train_val_data = ParaCrawl.load_text_pairs(os.path.join(base_path, "paracrawl_enpt_train.tsv.gz"))

        random.shuffle(train_val_data)

        train_data = train_val_data[:train_size]
        val_data = train_val_data[train_size:train_size + val_size]

        for mode, data in zip(ParaCrawl.VALID_MODES, [train_data, val_data, test_data]):
            file_name = os.path.join(base_path, f"{mode}.pkl")
            with open(file_name, "wb") as pkl_file:
                joblib.dump(data, pkl_file)
            print(f"Pre-processed data saved as {file_name}.")


datasets = {m: ParaCrawl(mode=m, seq_len=hparams["seq_len"]) for m in ParaCrawl.VALID_MODES}

# Testing datasets
for mode, dataset in datasets.items():
    print(f"\n{mode} dataset length: {len(dataset)}\n")
    print("Random sample")
    input, target, input_ids, input_mask, target_ids, target_mask = random.choice(dataset)
    print("input\n", input, end="\n\n")
    print("target\n", target, end="\n\n")


train dataset length: 9997000

Random sample
input
 Parte do que o nplanejamento deve

target
 Parte do que o planejamento deve


validation dataset length: 2670

Random sample
input
 Obrigado por baixar Mesh to fRoof Tool INT da nossa base de dados de

target
 Obrigado por baixar Mesh to Roof Tool INT da nossa base de dados de


test dataset length: 200000

Random sample
input
 No Windows 95 ou 98, você usou o fdimage ou o rawrite em modo DOS? Esses sistnemas operacionais as

target
 No Windows 95 ou 98, você usou o fdimage ou o rawrite em modo DOS? Esses sistemas operacionais as



## Dataloaders

Verificação se dataloaders estão funcionando corretamente.

In [8]:
shuffle = {"train": True, "validation": False, "test": False}
debug_dataloaders = {mode: datasets[mode].get_dataloader(batch_size=hparams["batch_size"], 
                                                         shuffle=shuffle[mode])
                     for mode in ParaCrawl.VALID_MODES}

# Testing dataloaders
for mode, dataloader in debug_dataloaders.items():
    print("{} number of batches: {}".format(mode, len(dataloader)))
    batch = next(iter(dataloader))

train number of batches: 99970
validation number of batches: 27
test number of batches: 2000


## Lightning Module

Aqui a classe principal do PyTorch Lightning é definida.


In [9]:
class T5Corrector(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        self.t5 = T5ForConditionalGeneration.from_pretrained(hparams.model_name,
                                                             cache_dir=hparams.base_path)
        self.tokenizer = ParaCrawl.TOKENIZER
        self.start_token = ParaCrawl.TOKENIZER.convert_tokens_to_ids("<extra_id_0>")
        self.end_token = ParaCrawl.TOKENIZER.convert_tokens_to_ids("<extra_id_1>")

    def forward(self, x):
        inputs, targets, input_ids, input_mask, target_ids, target_mask = x

        if self.training:
            outputs = self.t5(input_ids=input_ids, 
                              attention_mask=input_mask,
                              lm_labels=target_ids)
            loss, predicted_scores = outputs[:2]
            return loss, predicted_scores, inputs, targets
        else:
            outputs = self.t5.generate(input_ids=input_ids, 
                                       attention_mask=input_mask,
                                       lm_labels=target_ids,
                                       max_length=self.hparams.seq_len)
            return outputs, inputs, targets

    def training_step(self, batch, targets):
        loss, predicted_scores, inputs, targets = self(batch)

        return {"loss": loss, "log": {"loss": loss}, "progress_bar": hardware_stats()}

    def validation_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)

        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            )
            for output_ids in outputs]

        with open("orig.txt", "w") as f:
            for input in inputs:
                input = input.replace("\n", "")
                f.write(f"{input}\n")

        with open("ref.txt", "w") as f:
            for target in targets:
                target = target.replace("\n", "")
                f.write(f"{target}\n")

        with open("hyp.txt", "w") as f:
            for pred in predicts:
                pred = pred.replace("\n", "")
                f.write(f"{pred}\n")

        !errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
        !errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null
        x = !errant_compare -hyp hyp.m2 -ref ref.m2
        df = pd.DataFrame(data=x[2:4])[0].str.split("\t", expand=True)
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:].apply(pd.to_numeric) #take the data less the header row
        df.columns = new_header

        true_positive = df["TP"][1]
        false_positive = df["FP"][1]
        false_negative = df["FN"][1]
        precision = df["Prec"][1]
        recall = df["Rec"][1]
        f_score = df["F0.5"][1]

        progress_bar = hardware_stats()
        progress_bar.update({"precision": precision, "recall": recall, "f_score": f_score})

        return {"true_positive": true_positive, "false_positive": false_positive, "false_negative": false_negative,
                "precision": precision, "recall": recall, "f_score": f_score,
                "predicts": predicts, "inputs": inputs, "targets": targets, "progress_bar": progress_bar}

    def test_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)

        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            )
            for output_ids in outputs]

        with open("orig.txt", "w") as f:
            for input in inputs:
                input = input.replace("\n", "")
                f.write(f"{input}\n")

        with open("ref.txt", "w") as f:
            for target in targets:
                target = target.replace("\n", "")
                f.write(f"{target}\n")

        with open("hyp.txt", "w") as f:
            for pred in predicts:
                pred = pred.replace("\n", "")
                f.write(f"{pred}\n")

        !errant_parallel -orig orig.txt -cor ref.txt -out ref.m2 > /dev/null
        !errant_parallel -orig orig.txt -cor hyp.txt -out hyp.m2 > /dev/null
        x = !errant_compare -hyp hyp.m2 -ref ref.m2
        df = pd.DataFrame(data=x[2:4])[0].str.split("\t", expand=True)
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:].apply(pd.to_numeric) #take the data less the header row
        df.columns = new_header

        true_positive = df["TP"][1]
        false_positive = df["FP"][1]
        false_negative = df["FN"][1]
        precision = df["Prec"][1]
        recall = df["Rec"][1]
        f_score = df["F0.5"][1]

        progress_bar = hardware_stats()
        progress_bar.update({"precision": precision, "recall": recall, "f_score": f_score})

        return {"true_positive": true_positive, "false_positive": false_positive, "false_negative": false_negative,
                "precision": precision, "recall": recall, "f_score": f_score,
                "predicts": predicts, "inputs": inputs, "targets": targets, "progress_bar": progress_bar}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        return {"log": {"train_loss": avg_loss}} 

    def validation_epoch_end(self, outputs):
        avg_precision = sum([x["precision"] for x in outputs]) / len(outputs)
        avg_recall = sum([x["recall"] for x in outputs]) / len(outputs)
        avg_f_score = sum([x["f_score"] for x in outputs]) / len(outputs)

        tensorboard_logs = {"val_loss": avg_f_score,
                            "avg_precision": avg_precision,
                            "avg_recall": avg_recall,
                            "avg_f_score": avg_f_score}

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")

        return {"val_loss": avg_f_score, "avg_precision": avg_precision, "avg_recall": avg_recall, "avg_f_score": avg_f_score,
                "log": tensorboard_logs, "progress_bar": tensorboard_logs}

    def test_epoch_end(self, outputs):
        avg_precision = sum([x["precision"] for x in outputs]) / len(outputs)
        avg_recall = sum([x["recall"] for x in outputs]) / len(outputs)
        avg_f_score = sum([x["f_score"] for x in outputs]) / len(outputs)

        tensorboard_logs = {"avg_precision": avg_precision,
                            "avg_recall": avg_recall,
                            "avg_f_score": avg_f_score}

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")
        
        return {"avg_precision": avg_precision, "avg_recall": avg_recall, "avg_f_score": avg_f_score,
                "log": tensorboard_logs, "progress_bar": tensorboard_logs}

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.hparams.lr)    

    def train_dataloader(self):
        if self.hparams.overfit_pct > 0:
            logging.info("Disabling train shuffle due to overfit_pct.")
            shuffle = False
        else:
            shuffle = True
        dataset = ParaCrawl("train", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=shuffle)

    def val_dataloader(self):
        dataset = ParaCrawl("validation", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=False)

    def test_dataloader(self):
        dataset = ParaCrawl("test", seq_len=self.hparams.seq_len)
        return dataset.get_dataloader(batch_size=self.hparams.batch_size, shuffle=False)

## Preparação e Treino
Define-se hiperparâmetros para um treinamento de melhora, agora com decodificação corrigida.

In [10]:
hparams = {"name": experiment_name, "base_path": base_path,
           "model_name": model_name, "seq_len": sequence_length,
           "decode_mode": decode_mode, "k": k,
           "lr": learning_rate, "batch_size": batch_size, "batch_accum": accumulate_grad_batches,
           "max_epochs": max_epochs,
           "overfit_pct": 0, "debug": 0,
           "decode_mode": decode_mode}


for key, parameter in hparams.items():
    print("{}: {}".format(key, parameter))

name: adversarial-misspellings
base_path: /content/paracrawl-t5
model_name: t5-small
seq_len: 100
decode_mode: topk
k: 10
lr: 0.005
batch_size: 100
batch_accum: 1
max_epochs: 1
overfit_pct: 0
debug: 0


In [11]:
# Instantiate model
model = T5Corrector(Namespace(**hparams))

# Folder/path management, for logs and checkpoints
tensorboard_path = os.path.join(base_path, "logs")
experiment_name = hparams["name"]
model_folder = os.path.join(tensorboard_path, experiment_name)
os.makedirs(model_folder, exist_ok=True)
ckpt_path = os.path.join(model_folder, "-{epoch}")

# Callback initialization
checkpoint_callback = ModelCheckpoint(prefix=experiment_name, 
                                      filepath=ckpt_path, 
                                      mode="max",
                                      verbose=True)
logger = TensorBoardLogger(tensorboard_path, experiment_name, version=0)

# PL Trainer initialization
trainer = Trainer(gpus=1,
                  checkpoint_callback=checkpoint_callback, 
                  early_stop_callback=False,
                  logger=logger,
                  accumulate_grad_batches=hparams["batch_accum"],
                  max_epochs=hparams["max_epochs"], 
                  fast_dev_run=bool(hparams["debug"]), 
                  overfit_pct=hparams["overfit_pct"],
                  progress_bar_refresh_rate=10,
                  val_check_interval=0.1, # check validation set 10 times during a training epoch
                  profiler=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


## Tensorboard

In [12]:
%load_ext tensorboard
%tensorboard --logdir "/content/paracrawl-t5"
# %tensorboard --logdir "/content/drive/My Drive/PF-Correcao/paracrawl-t5"

Reusing TensorBoard on port 6006 (pid 8564), started 0:03:24 ago. (Use '!kill 8564' to kill it.)

## Fine-tuning

In [ ]:
trainer.fit(model)


  | Name | Type                       | Params
----------------------------------------------------
0 | t5   | T5ForConditionalGeneration | 60 M  


\Input: Os atributos do elemento <phpunit> podem ser usados paera configurar a funcionalidade principal do
Target: Os atributos do elemento <phpunit> podem ser usados para configurar a funcionalidade principal do
Prediction: <extra_id_0> elemento  ⁇ phpunit> podem ser usados paera configurar a funcionalidade principal do elemento  ⁇ phpunit>.



/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: The metric you returned 0.2694296296296296 must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of val_loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)

Epoch 00001: val_loss reached 0.26943 (best 0.26943), saving model to /content/paracrawl-t5/logs/adversarial-misspellings/adversarial-misspellings-epoch=1.ckpt as top 1


\Input: SYREN: Guitarrista anuncian parceria com a AMT Electronics | Metal
Target: SYREN: Guitarrista anuncia parceria com a AMT Electronics | Metal
Prediction: SYREN: Guitarrista anuncia parceria com a AMT Electronics | Metal



\Input: A IMPROVEMENT oferece para os seus cliemntes, uma equipe de profissionais altamente qualificados,
Target: A IMPROVEMENT oferece para os seus clientes, uma equipe de profissionais altamente qualificados,
Prediction: A IMPROVEMENT oferece para os seus clientes, uma equipe de profissionais altamente qualificados,



\Input: E sua natureza é tão inteiramente a de Deus, yque é a mesma imagem da substância de
Target: E sua natureza é tão inteiramente a de Deus, que é a mesma imagem da substância de
Prediction: E sua natureza é tão inteiramente a de Deus, que é a mesma imagem da substância de



\Input: Prefiro o café sem yleite e
Target: Prefiro o café sem leite e
Prediction: Prefiro o café sem leite e



\Input: “Ao anjo da igreja em Filadélfia escreve: Estas coisas diz o santo, o verdaadeiro, aquele que tem a
Target: “Ao anjo da igreja em Filadélfia escreve: Estas coisas diz o santo, o verdadeiro, aquele que tem a
Prediction: “Ao anjo da igreja em Filadélfia escreve: Estas coisas diz o santo, o verdadeiro, aquele que tem a



\Input: Aqui vemos a Ceristo dizendo efetivamente mediante linguagem simbólica que este é o tipo de amor que
Target: Aqui vemos a Cristo dizendo efetivamente mediante linguagem simbólica que este é o tipo de amor que
Prediction: Aqui vemos a Cristo dizendo efetivamente mediante linguagem simbólica que este é o tipo de amor que



\Input: Este é o programa que o santo Bispo entrega aos anunciadoees da Palavra ontem e hoje um programa
Target: Este é o programa que o santo Bispo entrega aos anunciadores da Palavra ontem e hoje um programa
Prediction: Este é o programa que o santo Bispo entrega aos anunciadores da Palavra ontem e hoje um programa



## Testes

In [ ]:
trainer.test(model)